# Analysis

## Setup

Zuerst laden wir die benötigten Daten herunter und initialisieren die genutzten Python Objekte.

In [ ]:
tables = [
    "careplans",
    "conditions",
    "observations",
    "patients",
]

files = [
    "data/others/",
    "data/asthma/",
    "data/gallstones/",
    "data/hypertension/",
]

In [ ]:
!mkdir -p data/allergy

In [ ]:
from urllib.request import urlopen
import os

def ensure_file_has_been_downloaded(filename):
    full_filename = "../" + filename

    url = "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/" + filename
    if os.path.isfile(full_filename):
        print("File {} already exists, skipping download".format(filename))
    else:
        print("Downloading {}".format(filename))
        download_file(url, full_filename)

def download_file(url, filename):
    with open(filename, 'wb') as out_file:
        with urlopen(url) as file:
            out_file.write(file.read())

if not os.path.isfile("extract.py"):
    download_file(
        "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/src/extract.py",
        "extract.py"
    )

for file in files:
    for table in tables:
        ensure_file_has_been_downloaded(file+table+".csv")

In [ ]:
from logger import Logger
from test_executer import TestExecutor

logger = Logger()
testExecutor = TestExecutor(logger)

## Mit Datenbank verbinden

In [ ]:
import extract
import time

databaseFile = "data.sqlite"

logger.startTimeMeasurement('open-db', 'Connected to db and created tables')
connection = extract.connect_to_db(logger, databaseFile)  # create table patients, observations, conditions, careplans
logger.endTimeMeasurement('open-db')

In [ ]:
def test_sqliteConnection(_logger):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tablesInDb = list(map(lambda tableResult: tableResult[0], cursor.fetchall()))
    tablesInDb.sort()

    for table in tables:
        if not(table in tablesInDb):
            raise Exception('Table not found:', table)

testExecutor.execute('Test connection to database', test_sqliteConnection)

## Daten in Datenbank laden

Lade der verwendete Daten in die Datenbank:

-   careplans
-   conditions
-   observations
-   patients

In [ ]:
logger.startTimeMeasurement('load-data', 'Loading data into db')
for file in files:
    for table in tables:
        extract.insert_values_to_table(logger, connection.cursor(), table, "../"+ file + table + ".csv")  # TODO: insert ALL values in the right tables 
        connection.commit()
    logger.endTimeMeasurement('load-data')

# Annonymisierung

In [ ]:
import pandas as pd
import random
import hashlib

patientDf = pd.read_sql_query('SELECT * FROM patients;', connection)
careplansDf = pd.read_sql_query('SELECT * FROM careplans;', connection)
conditionsDf = pd.read_sql_query('SELECT * FROM conditions;', connection)
observationsDf = pd.read_sql_query('SELECT * FROM observations;', connection)

patientIds = [*patientDf.Id, *careplansDf.PATIENT, *conditionsDf.PATIENT, *observationsDf.PATIENT]

def test_sanityCheckCombiningIds(_logger): 
    expectedLen = len(patientDf) + len(careplansDf) + len(conditionsDf) + len(observationsDf)
    actualLen = len(patientIds)
    if actualLen != expectedLen:
        raise Exception('Not all patient ids were concatenated')

testExecutor.execute('Sanity check: extracting all ids worked', test_sanityCheckCombiningIds)

# Converts list to a set with only unique values
uniqueIds = set(patientIds)
logger.log(f"{len(uniqueIds)} unique patient ids found")
if len(uniqueIds) >= len(patientDf.Id):
    logger.log(f"⚠️ The dataset contains {len(uniqueIds)} unique patientIds but only {len(patientDf.Id)} patients.")

annonymizedIds = {}
for id in uniqueIds:
    # Use uppercase here so it is easy to see if annonymized ids are used.
    annonymizedIds[id] = hashlib.sha256(f"{id}={random.random()}".encode()).hexdigest().upper()

def test_sanityEnsureAllIdsAreAnnonymized(_logger): 
    for id in patientIds:
        if id in annonymizedIds:
            raise Exception('A origin id still exists in anonnymized id list')

testExecutor.execute('Sanity check: no origin ids exist anymore', test_sanityCheckCombiningIds)

logger.startTimeMeasurement('annonymizedPatients', 'Writing annonymized patients')
patientDf = patientDf.replace({"Id": annonymizedIds})
patientDf.to_sql(name="anonnymized_patients", con=connection, if_exists='replace')
logger.endTimeMeasurement('annonymizedPatients')

logger.startTimeMeasurement('annonymizedCareplans', 'Writing annonymized careplans')
careplansDf = careplansDf.replace({"PATIENT": annonymizedIds})
careplansDf.to_sql(name="anonnymized_careplans", con=connection, if_exists='replace')
logger.endTimeMeasurement('annonymizedCareplans')

logger.startTimeMeasurement('annonymizedConditions', 'Writing annonymized conditions')
conditionsDf = conditionsDf.replace({"PATIENT": annonymizedIds})
conditionsDf.to_sql(name="anonnymized_conditions", con=connection, if_exists='replace')
logger.endTimeMeasurement('annonymizedConditions')

logger.startTimeMeasurement('annonymizedObservations', 'Writing annonymized Observations')
observationsDf = observationsDf.replace({"PATIENT": annonymizedIds})
observationsDf.to_sql(name="anonnymized_observations", con=connection, if_exists='replace')
logger.endTimeMeasurement('annonymizedObservations')


## Messung der Datenfehler

Für unsere Forschungsfrage sind nur alle Daten mit gemessenen BMI relevant. Wurde dieser nicht vermessen oder eingetragen können die Daten für die Forschungsfrage nicht verwendet werden und sind somit unbrauchbar.

In [ ]:
all_patients_query = """
SELECT COUNT(id) FROM patients;"""
count_bmi_query = """
SELECT COUNT(distinct id) FROM patients JOIN observations on patients.id == observations.patient WHERE observations.Code = '59576-9'"""

count_all_bmi_query = f"""
SELECT COUNT(patient) FROM observations WHERE observations.Code = '59576-9'"""

patient_all_count = connection.execute(all_patients_query).fetchall()[0][0]
patient_bmi_count = connection.execute(count_bmi_query).fetchall()[0][0]
bmi_count = connection.execute(count_all_bmi_query).fetchall()[0][0]
ratio = round(patient_bmi_count/patient_all_count, 3) * 100

logger.log(f"Total num of patients {patient_all_count}.")
logger.log(f"Found {patient_bmi_count} patients ({ratio}%) with {bmi_count} BMI-values.")
if len(uniqueIds) != patient_all_count:
    logger.log(f"The dataset contains further patient ids not present in the patient table. Amount of patients {patient_all_count} - Unique patient ids {len(uniqueIds)}")

In [ ]:
cursor = connection.cursor()

In [ ]:
# print tables of the database
cursor.execute('SELECT name FROM sqlite_master where type="table"') 
print(cursor.fetchall())

# Star Schema

#### Erstellen Dimensionstabellen

Tabelle patients_

In [ ]:
# table patients_
cursor.execute('''DROP TABLE IF EXISTS patients_;''')
cursor.execute('''
        CREATE TABLE patients_ ( 
        ID STRING PRIMARY KEY UNIQUE,
        BIRTHDATE DATE,
        RACE STRING, 
        ETHNICITY STRING,
        GENDER STRING
        );''')
   
cursor.execute('INSERT INTO patients_ (ID, BIRTHDATE, RACE, ETHNICITY, GENDER) SELECT ID, BIRTHDATE, RACE, ETHNICITY, GENDER FROM PATIENTS;')

print(pd.read_sql_query("PRAGMA table_info(patients_)", connection))

Tabelle observations_

In [ ]:
# table observations_
cursor.execute('''DROP TABLE IF EXISTS observations_;''')
cursor.execute('''
        CREATE TABLE observations_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING,
        UNITS STRING, 
        TYPE STRING
        );''')

# create df observation
cursor.execute('''SELECT CODE, DESCRIPTION, UNITS, TYPE FROM OBSERVATIONS;''')
df_observations = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION', 'UNITS', 'TYPE'])

df_observations = df_observations.drop_duplicates(subset='CODE')

# transform dt in table
df_observations.to_sql('df_observations', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO observations_ (CODE, DESCRIPTION, UNITS, TYPE) SELECT CODE, DESCRIPTION, UNITS, TYPE FROM df_observations;''')

cursor.execute('''DROP TABLE IF EXISTS df_observations;''')
print(pd.read_sql_query("PRAGMA table_info(observations_)", connection))
#print(pd.read_sql_query("SELECT * FROM observations_", connection))

Tabelle careplans_code

In [ ]:
# table careplans_code
cursor.execute('''DROP TABLE IF EXISTS careplans_code;''')
cursor.execute('''
        CREATE TABLE careplans_code ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT CODE, DESCRIPTION FROM CAREPLANS;''')
df_careplans_code = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_careplans_code = df_careplans_code.drop_duplicates(subset='CODE')

# transform dt in table
df_careplans_code.to_sql('df_careplans_code', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO careplans_code (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_careplans_code;''')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_code;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_code)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_code", connection))

Tabelle careplans_reasoncode

In [ ]:
# table careplans_reasoncode
cursor.execute('''DROP TABLE IF EXISTS careplans_reasoncode;''')
cursor.execute('''
        CREATE TABLE careplans_reasoncode ( 
        REASONCODE STRING PRIMARY KEY UNIQUE,
        REASONDESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT REASONCODE, REASONDESCRIPTION FROM CAREPLANS;''')
df_careplans_reasoncode = pd.DataFrame(cursor.fetchall(), columns=['REASONCODE','REASONDESCRIPTION'])

df_careplans_reasoncode = df_careplans_reasoncode.drop_duplicates(subset='REASONCODE')

# transform dt in table
df_careplans_reasoncode.to_sql('df_careplans_reasoncode', connection, if_exists='replace', index=False)
   
cursor.execute('INSERT INTO careplans_reasoncode (REASONCODE, REASONDESCRIPTION) SELECT REASONCODE, REASONDESCRIPTION FROM df_careplans_reasoncode;')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_reasoncode;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_reasoncode)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_reasoncode", connection))

Tabelle conditions_

In [ ]:
# table conditions
cursor.execute('''DROP TABLE IF EXISTS conditions_;''')
cursor.execute('''
        CREATE TABLE conditions_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df conditions
cursor.execute('''SELECT CODE, DESCRIPTION FROM CONDITIONS;''')
df_conditions = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_conditions = df_conditions.drop_duplicates(subset='CODE')

# transform dt in table
df_conditions.to_sql('df_conditions', connection, if_exists='replace', index=False)

cursor.execute('INSERT INTO conditions_ (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_conditions;')

cursor.execute('''DROP TABLE IF EXISTS df_conditions;''')
print(pd.read_sql_query("PRAGMA table_info(conditions_)", connection))
#print(pd.read_sql_query("SELECT * FROM conditions_", connection))

#### Erstelle Faktentabelle

In [ ]:
cursor.execute('''DROP TABLE IF EXISTS fact_table;''')
cursor.execute('''
        CREATE TABLE fact_table ( 
        PATIENT_ID STRING,
        OBSERVATION_CODE STRING,
        VALUE STRING,
        DATE DATE,
        CONDITIONS_CODE STRING,
        ENDDATE DATE,
        CAREPLANS_CODE STRING,
        CAREPLANS_REASONCODE STRING,
        FOREIGN KEY (PATIENT_ID)
            REFERENCES patients_(ID),
        FOREIGN KEY (OBSERVATION_CODE)
            REFERENCES observations_(CODE),
        FOREIGN KEY (CONDITIONS_CODE)
            REFERENCES conditions_(CODE),
        FOREIGN KEY (CAREPLANS_CODE)
            REFERENCES careplans_code(CODE),
        FOREIGN KEY (CAREPLANS_REASONCODE)
            REFERENCES careplans_reasoncode(REASONCODE)
        );''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, OBSERVATION_CODE, VALUE, DATE) 
                    SELECT PATIENT, CODE, VALUE, DATE 
                    FROM OBSERVATIONS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_REASONCODE, DATE, ENDDATE) 
                    SELECT PATIENT, REASONCODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CONDITIONS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CONDITIONS
                    ;''')

In [ ]:
print(pd.read_sql_query("PRAGMA foreign_key_list(fact_table)", connection))

In [ ]:
cursor.execute('SELECT name FROM sqlite_master where type="table"')
print(cursor.fetchall())

## Analysis

In [ ]:
print(pd.read_sql_query("SELECT reasoncode FROM careplans_reasoncode WHERE reasondescription='Diabetes'", connection))

In [ ]:
print(pd.read_sql_query('''SELECT fact_table.patient_ID
                        FROM fact_table
                        WHERE careplans_reasoncode=
                        (SELECT reasoncode FROM careplans_reasoncode WHERE reasondescription='Diabetes' OR reasondescription='Prediabetes')
                        ''', connection))

## Aufräumen & Logs speichern

In [ ]:
connection.close()
#logger.logTimings()
#logger.writeToFile("../artefacts-for-release/analysis-log.json")